# CFRP Data Analysis

This is where we will clean and format the data from the CFRP API to use in our data visualization.

In [9]:
# imports
import pandas as pd
import json
import tabulate
from IPython.display import HTML, display

## Data format

The given data format from the endpoint `http://api.cfregisters.org/play_ticket_sales` with params `genre=genre=eq.comédie` and `play_performance_id=lt.1000`

In [10]:
{
    "play_performance_id": 437, 
    "seating_capacity": 6,
    "name": "Premières Places",
    "title": "Avocat Patelin (L')",
    "author": "Brueys (David-Augustin de)",
    "genre": "comédie",
    "date": "1779-03-09",
    "total_sold": 35
}

{'author': 'Brueys (David-Augustin de)',
 'date': '1779-03-09',
 'genre': 'comédie',
 'name': 'Premières Places',
 'play_performance_id': 437,
 'seating_capacity': 6,
 'title': "Avocat Patelin (L')",
 'total_sold': 35}

### What we need:

Because our interactive visualization (at least the demo) allows the user to scrub through monthly ticket sales data for each play, we need to aggregate the total amount of tickets sold for each seating area for each play for each month of the year (if possible). 

*pic here of visualization mock up*

Not every play is shown every month, so we will calculate the **most popular plays** for **each month** in a given year, and visualize the **sales data for each seating area** for those given plays.

### Process

We will take the JSON response from the API and turn it into a Pandas data frame for easy aggregation. We will follow a 3 step process to get our data into the format we need.

So first we will convert the JSON format as seen above into a CSV like this:

**[play_performance_id,title,author,name,date,total_sold]**

Then, after grouping by month and play, then averaging the sales data, we will use this format in the visualization:

**[title, seating_area_1, seating_area_2, seating_area_3, seating_area_4, seating_area_5]**

Where each row is the month of the year.

## Just show me the code

In [154]:
# Import the JSON data and convert to pandas data frame
with open('Data/CFRP-data.json', 'r') as f:
    data = json.load(f)

# create the data frame
df = pd.DataFrame(data)

In [155]:
# convenience dictionary for printing
int2mon = {
    1: "January",
    2: "February",
    3: "March",
    4: "April",
    5: "May",
    6: "June",
    7: "July",
    8: "August",
    9: "September",
    10: "October",
    11: "November",
    12: "December"
}

In [156]:
def parse_date(date):
    """ 
    Converts a string formatted date in an easy-to-access date array.
    
        Example input: "1776-07-04"
        Example output: [1776, 7, 4]
    """
    
    #splits the input date string into an array
    inp = date.split('-')
    
    # converts each element into an integer
    formatted = [int(d) for d in inp]
    
    return formatted

In [157]:
"""
    we need to get the most popular show from each month for each year
    steps:
    1. group by month and year
    
        aggregate by date
        
        for each date:
            convert to parsed object
            access month
"""

# Conver the string into date object
formatted_date_col = df['date'].apply(parse_date)

# grab only the month from the new date column
months_col = formatted_date_col.apply(lambda d: d[1])
years_col = formatted_date_col.apply(lambda d: d[0])

# rename the column to months
df.rename(columns={'date': 'month'}, inplace=True)

# apply the transformed columns
df['month'] = months_col
df['year'] = years_col

In [158]:
df

,author,month,genre,name,play_performance_id,seating_capacity,title,total_sold,year
0,Brueys (David-Augustin de),3,comédie,Premières Places,437,6,Avocat Patelin (L'),35,1779
1,Brueys (David-Augustin de),3,comédie,Secondes Places,437,6,Avocat Patelin (L'),62,1779
2,Brueys (David-Augustin de),3,comédie,Places de Parterre,437,6,Avocat Patelin (L'),309,1779
3,Brueys (David-Augustin de),3,comédie,Petites Loges,437,6,Avocat Patelin (L'),2,1779
4,Brueys (David-Augustin de),3,comédie,Troisièmes Places,437,6,Avocat Patelin (L'),30,1779
5,Molière (Jean-Baptiste Poquelin dit) - arrangé...,3,comédie,Premières Places,436,6,Festin de Pierre (Le),35,1779
6,Molière (Jean-Baptiste Poquelin dit) - arrangé...,3,comédie,Secondes Places,436,6,Festin de Pierre (Le),62,1779
7,Molière (Jean-Baptiste Poquelin dit) - arrangé...,3,comédie,Places de Parterre,436,6,Festin de Pierre (Le),309,1779
8,Molière (Jean-Baptiste Poquelin dit) - arrangé...,3,comédie,Petites Loges,436,6,Festin de Pierre (Le),2,1779
9,Molière (Jean-Baptiste Poquelin dit) - arrangé...,3,comédie,Troisièmes Places,436,6,Festin de Pierre (Le),30,1779


In [159]:
# each year in the dataset
years = df.year.unique()

In [186]:
"""
    will contain a dataframe for each year of data, each one organized by month
    
    export as 3 different csv files, one for each year
    
    within each file:
"""

# all data
data_by_years = {}

# most popular data
most_popular_data = {}

# set the year keys and the relevant dataframes
for y in years:
    data_by_years[y] = df[df['year'] == y].copy()
    
    # set empty objects
    most_popular_data[y] = {}

# group each year's data by month and play
# find the most popular play each month

# first, go through each year
for year in data_by_years:
    
    data = data_by_years[year]
    
    # separate each year's data bymonth
    months = data.groupby('month', as_index=False)
    
    print("*******************")
    print("Year: {}".format(year))
    print("*******************\n")
    
     # Store the most popular play object
    most_popular_play = {}
    max_sales = 0
        
    for m in months:
        
        month = m[0]
        print("Month: {}".format(int2mon[month]))
        
        # separate each month by the plays
        plays = m[1].groupby('title', as_index=False)
        
        
        # Find the most popular play
        for p in plays:
            
#             print(p[0])
            seat_types = p[1].groupby('name', as_index=False)
            total_sales = p[1]['total_sold'].sum()
            
            if total_sales > max_sales:
                max_sales = total_sales
                
                # set the most popular play object based on the total sales
                most_popular_play = p
        
#         print("Most popular play: {}".format(most_popular_play[0]))
        # Store the most popular data formatted by seating area name for each month
        most_popular_data[year][month] = most_popular_play[1].groupby('name')
    print("Most popular play: {}".format(most_popular_play[0]))

print("\n\n\n")

*******************
Year: 1779
*******************

Month: January
Month: February
Month: March
Month: April
Month: May
Month: June
Month: July
Month: December
Most popular play: Muses rivales ou l'Apothéose de Voltaire (Les)
*******************
Year: 1780
*******************

Month: January
Month: February
Month: March
Most popular play: Étrennes de l'amitié, de l'amour et de la nature (Les)
*******************
Year: 1778
*******************

Month: January
Month: February
Month: March
Month: April
Month: May
Month: June
Most popular play: Homme personnel (L')






In [161]:
for year in most_popular_data:
    
    print("Year: {}".format(year))
    yearly_data = most_popular_data[year]

    for month in yearly_data.keys():
        
        print("Month: ", int2mon[month])
        
        for area_data in yearly_data[month]:
            
            print("Seating area: {}".format(area_data[0]))
#             print("Tickets sold: {}".format(area_data[1]['total_sold']))
            print("\n")
        
        print("\n\n")

Year: 1779
Month:  January
Seating area: Petites Loges


Seating area: Places de Parterre


Seating area: Premières Places


Seating area: Secondes Loges 3


Seating area: Secondes Places


Seating area: Troisièmes Places





Month:  February
Seating area: Petites Loges


Seating area: Places de Parterre


Seating area: Premières Loges 1


Seating area: Premières Loges 2


Seating area: Premières Loges 3


Seating area: Premières Loges 4


Seating area: Premières Places


Seating area: Secondes Loges 2


Seating area: Secondes Loges 3


Seating area: Secondes Places


Seating area: Troisièmes Loges


Seating area: Troisièmes Places





Month:  March
Seating area: Petites Loges


Seating area: Places de Parterre


Seating area: Premières Loges 2


Seating area: Premières Loges 4


Seating area: Premières Places


Seating area: Secondes Loges 1


Seating area: Secondes Loges 2


Seating area: Secondes Loges 3


Seating area: Secondes Places


Seating area: Troisièmes Places





Month:

### Collecting data from specific play, not just most popular (because of differences in seating arrangements...)

In [192]:
"""
    will contain a dataframe for each year of data, each one organized by month
    export as 3 different csv files, one for each year
    within each file:
"""

from collections import defaultdict

# all data
data_by_years = {}

#export dict
data_to_export = {}

# set the year keys and the relevant dataframes
for y in years:
    data_by_years[y] = df[df['year'] == y].copy()
    data_to_export[int(y)] = {}

# group each year's data by month and play
# find the most popular play each month

# first, go through each year
for year in data_by_years:
    
    print("*******************")
    print("Year: {}".format(year))
    print("*******************\n")
    
    data = data_by_years[year]
    
    #separate by plays
    plays = data.groupby('title', as_index=False)    
    
    for p in plays:
        if p[0] == "Muses rivales ou l'Apothéose de Voltaire (Les)":
            monthly_data = p[1].groupby('month')
            for m in monthly_data:
                print(m[0])
        
#     for m in months:
        
#         month = m[0]
#         plays = m[1].groupby('title', as_index=False)
        
            
#             # set the indices:
#         for p in plays:
#             data_to_export[int(year)][month] = {}
        
#         # Find the most popular play
#         for p in plays:
            
#             play_name = p[0]
#             seat_types = p[1].groupby('name', as_index=True)
            
#             relevant_seat_data = {}
            
#             for d in seat_types:            
#                 relevant_seat_data[d[0]] = d[1]['total_sold']
            
#             data_to_export[int(year)][month][play_name] = relevant_seat_data   
        
        # For collecting every play's data.
        
#         # set the indices:
#         for p in plays:
#             data_to_export[int(year)][month] = {}
        
#         # Find the most popular play
#         for p in plays:
            
#             play_name = p[0]
#             seat_types = p[1].groupby('name', as_index=True)
            
#             relevant_seat_data = {}
            
#             for d in seat_types:            
#                 relevant_seat_data[d[0]] = d[1]['total_sold']
            
#             data_to_export[int(year)][month][play_name] = relevant_seat_data

        ##Collecting a specific play's data 
    

*******************
Year: 1779
*******************

2
3
*******************
Year: 1780
*******************

*******************
Year: 1778
*******************

